In [126]:
# IMPORTS
# ---------
# Stop warnings
import warnings
warnings.filterwarnings("ignore")

# General imports
import os
import sys
import json
import glob
import numpy as np
import h5py
import pandas as pd
opj = os.path.join
import pdb

# Plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.offline as pxo

In [206]:
# DATA
# ----
# Get inputs
subject = 'sub-001'
task = 'GazeCenterFS'
preproc = 'fmriprep_dct_pca'

# Define analysis parameters
with open('mri_analysis/settings.json') as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)

base_dir = analysis_info['base_dir']
h5_dir = "{base_dir}/pp_data/{subject}/gauss/h5".format(base_dir = base_dir, subject = subject)

# load deriv data
rsq_idx, ecc_idx, polar_real_idx, polar_imag_idx , size_idx, \
    amp_idx, baseline_idx, cov_idx, x_idx, y_idx, hemi_idx = 0,1,2,3,4,5,6,7,8,9,10
rsq_dict, ecc_dict, polar_real_dict, polar_imag_dict, size_dict, x_dict, y_dict, hemi_dict = {}, {}, {}, {}, {}, {}, {}, {}
ecc_sample_dict, size_sample_dict = {}, {}

df_raw = pd.DataFrame()
for roi in analysis_info['rois']:
    
    h5_file = h5py.File("{h5_dir}/{roi}_{task}_{preproc}.h5".format(h5_dir = h5_dir, roi = roi, task = task, preproc = preproc),'r')
    deriv_data = h5_file['{folder_alias}/derivatives'.format(folder_alias = 'pRF')]
    deriv_data = deriv_data[~np.isnan(deriv_data[:,rsq_idx]),:]
    df_roi = pd.DataFrame(deriv_data,columns = ['rsq','ecc','polar_real','polar_imag','size','amp','baseline','cov','x','y','hemi'])
    df_roi['roi']=[roi for x in range(df_roi.shape[0])]
    df_roi['subject']=[subject for x in range(df_roi.shape[0])]
    df_roi['task']=[task for x in range(df_roi.shape[0])]
    df_roi['preproc']=[preproc for x in range(df_roi.shape[0])]
    df_raw = pd.concat([df_raw,df_roi],ignore_index=True,axis = 0)

#df.to_csv('{subject}_{task}_{preproc}.gz'.format(subject=subject, task=task, preproc=preproc),compression='gzip', float_format='%.4f')

rsqr_th, size_th, ecc_th = analysis_info['rsqr_th'], analysis_info['size_th'], analysis_info['ecc_th']
df = df_raw[(df_raw.rsq >= rsqr_th) & (df_raw['size'] >= size_th[0]) & (df_raw['size'] <= size_th[1]) &  (df_raw.ecc >= ecc_th[0]) & (df_raw.ecc <= ecc_th[1])]


In [207]:
idx_sample = df.index[df.roi=='V2'][np.random.permutation(df[df.roi=='V2'].shape[0])[0:analysis_info['sample_num']].tolist()]

2748    1.242508
4255    0.834068
4062    1.847929
3332    0.800954
2398    1.238619
          ...   
2464    0.836507
4007    1.218046
3140    1.249831
2687    2.662186
3561    1.666151
Name: size, Length: 100, dtype: float32

,rsq,ecc,polar_real,polar_imag,size,amp,baseline,cov,x,y,hemi,roi,subject,task,preproc
2740,0.368637,4.888224,0.890772,-0.454451,0.959352,0.000009,-0.142992,1.000000,4.354292,-2.221458,1.0,V2,sub-001,GazeCenterFS,fmriprep_dct_pca
4458,0.768338,1.429469,-0.989559,0.144131,0.705656,0.000028,-0.314460,1.000000,-1.414543,0.206031,2.0,V2,sub-001,GazeCenterFS,fmriprep_dct_pca
4291,0.135654,1.173918,-0.422463,-0.906380,0.343790,0.000029,-0.067563,1.000000,-0.495936,-1.064016,2.0,V2,sub-001,GazeCenterFS,fmriprep_dct_pca
2430,0.293003,2.750769,0.655010,-0.755621,1.255844,0.000004,-0.120840,1.000000,1.801780,-2.078538,1.0,V2,sub-001,GazeCenterFS,fmriprep_dct_pca
3207,0.275843,2.621892,0.980369,0.197171,1.267947,0.000004,-0.133244,1.000000,2.570422,0.516960,1.0,V2,sub-001,GazeCenterFS,fmriprep_dct_pca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4457,0.137492,1.150308,-0.621104,0.783728,1.234847,0.000003,-0.097098,1.000000,-0.714461,0.901529,2.0,V2,sub-001,GazeCenterFS,fmriprep_dct_pca
2473,0.199795,3.020403,0.776939,-0.629575,1.210341,0.000005,-0.120160,1.000000,2.346670,-1.901572,1.0,V2,sub-001,GazeCenterFS,fmriprep_dct_pca
3101,0.524519,9.860304,0.577129,-0.816653,2.396744,0.000032,-0.297554,0.707673,5.690666,-8.052448,1.0,V2,sub-001,GazeCenterFS,fmriprep_dct_pca
3493,0.172515,9.827011,-0.361314,0.932444,1.449748,0.001188,-0.095712,0.616548,-3.550637,9.163139,2.0,V2,sub-001,GazeCenterFS,fmriprep_dct_pca


rsq                   0.576569
ecc                    3.71655
polar_real           -0.640362
polar_imag           -0.768073
size                   1.68178
amp                5.62076e-06
baseline             -0.262113
cov                   0.999994
x                     -2.37993
y                     -2.85458
hemi                         2
roi                         V3
subject                sub-001
task              GazeCenterFS
preproc       fmriprep_dct_pca
Name: 6691, dtype: object